In [1]:
# Parameters
outlier_rem = None
date = "2025_06_29"
derivative = None
files_date = None


# Pre-processing steps for Gap Overlap

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import time
import os
import statsmodels
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from scipy import stats
import pingouin as pg
import scikit_posthocs as sp
import statsmodels.api as sm
from scipy.stats import boxcox
from statannotations.Annotator import Annotator
from scipy.stats import pearsonr
import docx
from docx.enum.section import WD_ORIENT
from docx.enum.section import WD_SECTION
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [3]:
# Date 
date ="2025_06_27"

In [4]:
# Papermill parameters
date = "2025_06_27"
min_n_trials_per_condition = 6

In [5]:

def df_to_table(doc, df):
    # Add a table to the end and create a reference variable
    t = doc.add_table(df.shape[0]+1, df.shape[1])

    # Make sure df is rounded 
    df = df.round(2)

    # Make sure table is auto fit
    for cell in t._cells:
        cell.width = docx.shared.Inches(1.0)
    
    # Set the style of the table
    table_style = doc.styles['Table Grid']

    # Add the header rows.
    for j in range(df.shape[-1]):
        t.cell(0,j).text = df.columns[j]

    # Add the rest of the data frame
    for i in range(df.shape[0]):
        for j in range(df.shape[-1]):
            t.cell(i+1,j).text = str(df.values[i,j])
    
    return doc


#### Create a word preprocessing results

In [6]:

# Create a document

doc = docx.Document()

# Title page
section = doc.sections[0]
#section.start_type = WD_SECTION.ODD_PAGE
#section.orientation = WD_ORIENT.LANDSCAPE
#section.page_width, section.page_height = section.page_height, section.page_width

title = doc.add_paragraph()
title_run = title.add_run("Gap Overlap preprocessing results")
title_run.bold = True
title_run.font.size = Pt(28)
title_run.font.name = 'Arial'
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph("")
doc.add_paragraph(f"Date: {date}").alignment = WD_ALIGN_PARAGRAPH.CENTER
doc.add_paragraph("Author: Gabriel Blanco").alignment = WD_ALIGN_PARAGRAPH.CENTER
doc.add_page_break()

# Add new doc section with landscape orientation
section = doc.add_section()
section.start_type = WD_SECTION.ODD_PAGE
new_width, new_height = section.page_height, section.page_width
section.orientation = WD_ORIENT.LANDSCAPE
section.page_width = new_width


doc.add_heading(f"Data pre-processings results", 0)

doc.add_heading('Invalid Trial Criteria', level=1)
doc.add_paragraph(
    "A trial is considered invalid if it meets any of the following criteria:\n\n"
    "1. Did not look at fixation point for at least 500 ms\n"
    "   - The participant was distracted at the beginning of the trial and failed to maintain gaze on the fixation point for the required duration.\n\n"
    "2. Reaction time (RT) to peripheral target is greater than 1200 ms\n"
    "   - The participant took too long to respond, indicating possible distraction or inattention.\n\n"
    "3. Reaction time (RT) is less than 100 ms\n"
    "   - Extremely fast responses are likely anticipatory and not genuine reactions.\n\n"
    "4. Gaze directed to the wrong side of the screen\n"
    "   - The participant looked away from the intended target, indicating an incorrect response.\n\n"
    "5. Minimum number of trials\n"
    "   - The participant must have a minimum of 6 valid trials per condition in order to not be removed.\n\n"
    "Note:\n"
    "These criteria help ensure that only valid, attentive, and accurate trials are included in the analysis. "
    "Excluding invalid trials improves the reliability of behavioral and statistical results. "
    "Trial exclusion is automatically done by SR Research code and thus, invalid trials are coded into .txt files"
)


## Data loading

In [7]:
# Load et file
go_df= pd.read_csv("C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/outputs/gap/go_df.csv")
go_df        

,subject,trial_index,condition,side,rt,valid_trials,eeg_age,site,sex,asd,ndd,q1k_ID,group,age_group
0,002Z,1,Baseline,Right,230.31,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult
1,002Z,2,Gap,Right,255.00,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult
2,002Z,3,Overlap,Left,323.78,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult
3,002Z,4,Gap,Right,197.00,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult
4,002Z,5,Baseline,Left,372.62,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8249,1256S2,34,Overlap,Right,368.29,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult
8250,1256S2,35,Gap,Left,247.00,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult
8251,1256S2,36,Baseline,Right,211.76,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult
8252,1256S2,37,Overlap,Right,319.85,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult


In [8]:
# Load subjects removed file
subjects_removed_df = pd.read_csv("C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/outputs/missingness/subjects_removed.csv")
subjects_removed_df

,subject,status
0,004_Z,empty file
1,0105F1,empty file
2,1006P,empty file
3,1021F1,empty file
4,1045F1,empty file
5,1052P,empty file
6,1057F1,empty file
7,1089M1,empty file
8,1143S1,empty file
9,1147S2,empty file


In [9]:
# Print totl number of subjects and removed subjects
print(f"Total number of subjects: {len(go_df['subject'].unique())}")
print(f"Number of subjects removed: {len(subjects_removed_df['subject'].unique())}")

Total number of subjects: 230
Number of subjects removed: 12


## Calculate valid and invalid trials

## Invalid Trial Criteria

A trial is considered **invalid** if it meets any of the following criteria:

1. **Did not look at fixation point for at least 500 ms**  
    - The participant was distracted at the beginning of the trial and failed to maintain gaze on the fixation point for the required duration.

2. **Reaction time (RT) to peripheral target is greater than 1200 ms**  
    - The participant took too long to respond, indicating possible distraction or inattention.

3. **Reaction time (RT) is less than 100 ms**  
    - Extremely fast responses are likely anticipatory and not genuine reactions.

4. **Gaze directed to the wrong side of the screen**  
    - The participant looked away from the intended target, indicating an incorrect response.

5. **Minium number of trials**
    - The participnat must have a minimum of 6 valid trials per condition in order to not be removed

> **Note:**  
> These criteria help ensure that only valid, attentive, and accurate trials are included in the analysis. Excluding invalid trials improves the reliability of behavioral and statistical results. Trial exclusion is automatically done by SR Research code and thus, invalid trials are coded into .txt files


##### Remove trials under 0s

In [10]:
# Change reaction times below 0 to invalide trials 

# Print total number of trials that were removed
print(f"Total number of trials with quick negative RT: {len(go_df[go_df.rt < 0])}")
go_df.loc[go_df.rt < 0, "valid_trials"] = False

# Print total trials above 1200ms
print(f"Total number of trials with RT above 1200ms: {len(go_df[go_df.rt > 1200])}")

# Convert invalid trial's rt into np.nan values
go_df.loc[go_df.valid_trials == False, "rt"] = np.nan

Total number of trials with quick negative RT: 725
Total number of trials with RT above 1200ms: 186


#### Particpant statistics for accuracy 

In [11]:
# Calculate the overall number of trials and failed trials per subject into one dataframe
total_trials = go_df.groupby(["subject", "ndd"]).size().reset_index(name="total_trials")
failed_trials = go_df[go_df.valid_trials == False].groupby(["subject"]).size().reset_index(name="failed_trials")

# Calculate the numebr of failed "Overlap" trials
total_overlap = go_df[go_df.condition == "Overlap"].groupby(["subject"]).size().reset_index(name="total_overlap")
failed_overlap = go_df[(go_df.valid_trials == False) & (go_df.condition == "Overlap")].groupby(["subject"]).size().reset_index(name="failed_overlap")

# Calculate the numebr of failed "Baseline" trials
total_baseline = go_df[go_df.condition == "Baseline"].groupby(["subject"]).size().reset_index(name="total_baseline")
failed_baseline = go_df[(go_df.valid_trials == False) & (go_df.condition == "Baseline")].groupby(["subject"]).size().reset_index(name="failed_baseline")

# Calculate the numebr of failed "Gap" trials
total_gap = go_df[go_df.condition == "Gap"].groupby(["subject"]).size().reset_index(name="total_gap")
failed_gap = go_df[(go_df.valid_trials == False) & (go_df.condition == "Gap")].groupby(["subject"]).size().reset_index(name="failed_gap")

# Merge the all dataframes
trial_data = pd.merge(total_trials, failed_trials, on="subject", how="left")
trial_data = pd.merge(trial_data, total_overlap, on="subject", how="left")
trial_data = pd.merge(trial_data, failed_overlap, on="subject", how="left")
trial_data = pd.merge(trial_data, total_baseline, on="subject", how="left")
trial_data = pd.merge(trial_data, failed_baseline, on="subject", how="left")
trial_data = pd.merge(trial_data, total_gap, on="subject", how="left")
trial_data = pd.merge(trial_data, failed_gap, on="subject", how="left")

# Fill NaN values with 0
trial_data = trial_data.fillna(0)

# Percentage of failed trials
trial_data["failed_trials"] = trial_data["failed_trials"].fillna(0)
trial_data["%_failed"] = trial_data["failed_trials"] / trial_data["total_trials"] * 100

# Percentage of failed "Baseline" trials
trial_data["%_baseline"] = trial_data["failed_baseline"] / trial_data["total_baseline"] * 100

# Percentage of failed "Overlap" trials
trial_data["%_overlap"] = trial_data["failed_overlap"] / trial_data["total_overlap"] * 100

# Percentage of failed "Gap" trials
trial_data["%_gap"] = trial_data["failed_gap"] / trial_data["total_gap"] * 100

# Total valid trials 
trial_data["valid_trials"] = trial_data["total_trials"] - trial_data["failed_trials"]

# Total valid Baseline trials
trial_data["valid_baseline"] = trial_data["total_baseline"] - trial_data["failed_baseline"]

# Total valid Overlap trials
trial_data["valid_overlap"] = trial_data["total_overlap"] - trial_data["failed_overlap"]

# Total valid Gap trials
trial_data["valid_gap"] = trial_data["total_gap"] - trial_data["failed_gap"]

# Create column to highlight participants who have at least min 8 valid trials in all conditions
trial_data["valid_trials_all_conditions"] = (
    (trial_data["valid_baseline"] >= min_n_trials_per_condition) &
    (trial_data["valid_overlap"] >= min_n_trials_per_condition) &
    (trial_data["valid_gap"] >= min_n_trials_per_condition)
)

# Keep only important columns
trial_data = trial_data[["subject", "ndd", "total_trials", "failed_trials", "%_failed", "total_overlap", "%_overlap", "total_baseline", "%_baseline", "total_gap", "%_gap", 
"valid_trials", "valid_baseline", "valid_overlap", "valid_gap", "valid_trials_all_conditions"]] 

In [12]:
trial_data

,subject,ndd,total_trials,failed_trials,%_failed,total_overlap,%_overlap,total_baseline,%_baseline,total_gap,%_gap,valid_trials,valid_baseline,valid_overlap,valid_gap,valid_trials_all_conditions
0,002Z,no_ndd,36,1.0,2.777778,12.0,8.333333,12.0,0.000000,12.0,0.000000,35.0,12.0,11.0,12.0,True
1,0042P,ndd,36,1.0,2.777778,12.0,0.000000,12.0,8.333333,12.0,0.000000,35.0,11.0,12.0,12.0,True
2,0043F1,no_ndd,36,0.0,0.000000,12.0,0.000000,12.0,0.000000,12.0,0.000000,36.0,12.0,12.0,12.0,True
3,0050M1,no_ndd,36,1.0,2.777778,12.0,8.333333,12.0,0.000000,12.0,0.000000,35.0,12.0,11.0,12.0,True
4,0050P,ndd,44,11.0,25.000000,12.0,8.333333,16.0,31.250000,16.0,31.250000,33.0,11.0,11.0,11.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,1226P,ndd,30,15.0,50.000000,12.0,66.666667,4.0,25.000000,14.0,42.857143,15.0,3.0,4.0,8.0,False
226,1256F1,ndd,38,3.0,7.894737,12.0,0.000000,14.0,14.285714,12.0,8.333333,35.0,12.0,12.0,11.0,True
227,1256P,ndd,39,10.0,25.641026,13.0,46.153846,12.0,0.000000,14.0,28.571429,29.0,12.0,7.0,10.0,True
228,1256S1,ndd,36,1.0,2.777778,12.0,0.000000,12.0,8.333333,12.0,0.000000,35.0,11.0,12.0,12.0,True


In [13]:
# Add the trial data to the document
doc.add_heading('Vali trial analysis', level=1)

# Add bullet point for total number of participants 
doc.add_paragraph(f"Total number of participants: {len(trial_data['subject'].unique())}", style='ListBullet')

print(f"Total number of participants: {len(trial_data['subject'].unique())}")

# Add total number of participants with the min number of trials
doc.add_paragraph(f"Total number of participants with at least {min_n_trials_per_condition} valid trials in all conditions: {len(trial_data[trial_data['valid_trials_all_conditions'] == True])}", style='ListBullet')
print(f"Total number of participants with at least 6 valid trials in all conditions: {len(trial_data[trial_data['valid_trials_all_conditions'] == True])}")

Total number of participants: 230
Total number of participants with at least 6 valid trials in all conditions: 209


c:\Users\gabot\OneDrive - McGill University\Desktop\Github_repos\q1k_neurosubs\q1k_et_neurosubs_env\lib\site-packages\docx\styles\styles.py:125: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


In [14]:
# Add the subjects without min number of trial criteria to subjects_removed_df
missing_ids = trial_data[trial_data["valid_trials_all_conditions"] == False][["subject"]].copy()
# Add the subjects without min number of trial criteria to subjects_removed_dfm and add reasons as 'not enough trials'
missing_ids["status"] = f"not enough trials ({min_n_trials_per_condition})"
subjects_removed_df = pd.concat([subjects_removed_df, missing_ids], ignore_index=True)


In [15]:
subjects_removed_df

,subject,status
0,004_Z,empty file
1,0105F1,empty file
2,1006P,empty file
3,1021F1,empty file
4,1045F1,empty file
5,1052P,empty file
6,1057F1,empty file
7,1089M1,empty file
8,1143S1,empty file
9,1147S2,empty file


In [16]:
# Add column to go_df csv that flags subjects without enough trials

go_df["min_ntrials"] = go_df["subject"].isin(trial_data[trial_data["valid_trials_all_conditions"] == True]["subject"])
go_df


,subject,trial_index,condition,side,rt,valid_trials,eeg_age,site,sex,asd,ndd,q1k_ID,group,age_group,min_ntrials
0,002Z,1,Baseline,Right,230.31,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult,True
1,002Z,2,Gap,Right,255.00,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult,True
2,002Z,3,Overlap,Left,323.78,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult,True
3,002Z,4,Gap,Right,197.00,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult,True
4,002Z,5,Baseline,Left,372.62,True,45.88,mni,female,0.0,no_ndd,Q1K_002_Z,NaN,adult,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8249,1256S2,34,Overlap,Right,368.29,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult,True
8250,1256S2,35,Gap,Left,247.00,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult,True
8251,1256S2,36,Baseline,Right,211.76,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult,True
8252,1256S2,37,Overlap,Right,319.85,True,18.32,hsj,female,1.0,ndd,Q1K_HSJ_1525-1256_S2,sibling,adult,True


In [17]:
# Save go_df with the new column
go_df.to_csv("C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/outputs/gap/go_df.csv", index=False)

In [18]:
# Save the subjects removed df
subjects_removed_df.to_csv("C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/outputs/missingness/subjects_removed.csv", index=False)

### Demographics of subjects that need to be removed

In [19]:
# Select only subjects without min trials 
demo_df = go_df [go_df["min_ntrials"] == False]

# Select only the first row per participant
demo_df = demo_df.drop_duplicates(subset=["subject"], keep="first")
demo_df

,subject,trial_index,condition,side,rt,valid_trials,eeg_age,site,sex,asd,ndd,q1k_ID,group,age_group,min_ntrials
536,0083M1,1,Baseline,Left,NaN,False,40.95,hsj,female,NaN,ndd,Q1K_HSJ_10083_M1,mother,adult,False
1470,0147M1,1,Gap,Right,NaN,False,38.52,hsj,female,NaN,ndd,Q1K_HSJ_100147_M1,mother,adult,False
1582,0150M1,1,Gap,Left,NaN,False,37.90,hsj,female,NaN,ndd,Q1K_HSJ_100150_M1,mother,adult,False
1620,0152M1,1,Gap,Left,NaN,False,43.54,hsj,female,NaN,no_ndd,Q1K_HSJ_100152_M1,mother,adult,False
1630,0154P,0,Gap,Left,NaN,False,3.28,hsj,male,NaN,ndd,Q1K_HSJ_100154_P,proband,child,False
2159,0181P,1,Baseline,Right,306.99,True,20.14,mni,male,0.0,ndd,Q1K_MHC_200181_P,proband,adult,False
2749,0216P,1,Baseline,Right,190.26,True,14.43,mni,female,NaN,ndd,Q1K_MHC_200216_P,proband,child,False
3126,0248S1,1,Gap,Right,NaN,False,11.78,mni,male,NaN,ndd,Q1K_MHC_200248_S1,sibling,child,False
4375,1028P,1,Gap,Left,NaN,False,10.27,hsj,female,NaN,ndd,Q1K_HSJ_1525-1028_P,proband,child,False
4594,1037P,1,Gap,Left,NaN,False,6.56,hsj,male,NaN,ndd,Q1K_HSJ_1525-1037_P,proband,child,False


In [20]:
# Summarize data by diagnosis groups
summary_table = demo_df.groupby("ndd").agg(
    n=("ndd", "count"),
    male=("sex", lambda x: (x == "male").sum()),
    female=("sex", lambda x: (x == "female").sum()),
    mean_age=("eeg_age", "mean"),
    std_age=("eeg_age", "std"),
    hsj=("site", lambda x: (x == "hsj").sum()),
    mni=("site", lambda x: (x == "mni").sum()),
    adults=("age_group", lambda x: (x == "adult").sum()),
    children=("age_group", lambda x: (x == "child").sum()),
).reset_index()

# Add combined row for all participants
all_participants = {
    "ndd": "combined",
    "n": summary_table["n"].sum(),
    "male": summary_table["male"].sum(),
    "female": summary_table["female"].sum(),
    "mean_age": demo_df["eeg_age"].mean(),
    "std_age": demo_df["eeg_age"].std(),
    "hsj": summary_table["hsj"].sum(),
    "mni": summary_table["mni"].sum(),
    "adults": summary_table["adults"].sum(),
    "children": summary_table["children"].sum(),
}

summary_table = pd.concat(
    [pd.DataFrame([all_participants]), summary_table], ignore_index=True
)

# Create combined and percentage columns
summary_table["sex (M:F)"] = summary_table["male"].astype(str) + ":" + summary_table["female"].astype(str)
summary_table["sex % (M:F)"] = (
    (summary_table["male"] / summary_table["n"] * 100).round(2).astype(str)
    + ":"
    + (summary_table["female"] / summary_table["n"] * 100).round(2).astype(str)
)

summary_table["site (HSJ:MNI)"] = summary_table["hsj"].astype(str) + ":" + summary_table["mni"].astype(str)
summary_table["site % (HSJ:MNI)"] = (
    (summary_table["hsj"] / summary_table["n"] * 100).round(2).astype(str)
    + ":"
    + (summary_table["mni"] / summary_table["n"] * 100).round(2).astype(str)
)

# Drop intermediate columns to keep the table clean
summary_table.drop(columns=["male", "female", "hsj", "mni"], inplace=True)

# Rename columns for better clarity
summary_table.rename(
    columns={
        "ndd": "NDD Group",
        "n": "Total Participants",
        "mean_age": "Mean Age",
        "std_age": "Age SD",
        "adults": "Adults",
        "children": "Children",
    },
    inplace=True,
)

In [21]:
summary_table=summary_table.round(2)
summary_table

,NDD Group,Total Participants,Mean Age,Age SD,Adults,Children,sex (M:F),sex % (M:F),site (HSJ:MNI),site % (HSJ:MNI)
0,combined,21,20.24,16.96,8,13,13:8,61.9:38.1,18:3,85.71:14.29
1,ndd,16,21.43,17.68,7,9,11:5,68.75:31.25,13:3,81.25:18.75
2,no_ndd,5,16.41,15.56,1,4,2:3,40.0:60.0,5:0,100.0:0.0


In [22]:

## Add table to the document
doc.add_heading('Demographics breakdown of dropped participnats', level=1)

doc.add_heading('Demographics table', level=3)

doc = df_to_table(doc, summary_table)


## Calculate and describe the properties of invalid trials

In [23]:
# Check the proportion of overlap, baseline and gap in the number of invalid trials
doc.add_heading('Proportion of invalid trials', level=1)
# Calculate the proportion of invalid trials for each condition
proportion_invalid = trial_data[["subject", "ndd", "%_overlap", "%_baseline", "%_gap"]].copy()

# Describe proportion_invalid by calcualting the mean for only the dropped subjetcs

proportion_invalid = proportion_invalid[proportion_invalid["subject"].isin(subjects_removed_df["subject"])]

proportion_invalid = proportion_invalid.groupby("ndd").agg(
    mean_overlap=("%_overlap", "mean"),
     mean_baseline=("%_baseline", "mean"),
     mean_gap=("%_gap", "mean"),
 ).reset_index()
proportion_invalid = proportion_invalid.round(2)

# Add another column for the sum of total trials for each category only for those that were dropped

trial_data_dropped= trial_data[trial_data["subject"].isin(subjects_removed_df["subject"])]

proportion_invalid["summ_overlap"] = trial_data_dropped.groupby("ndd")["total_overlap"].sum().values
proportion_invalid["summ_baseline"] = trial_data_dropped.groupby("ndd")["total_baseline"].sum().values
proportion_invalid["summ_gap"] = trial_data_dropped.groupby("ndd")["total_gap"].sum().values


# Change column names to improve clarity
proportion_invalid.rename(
    columns={
        "ndd": "NDD Group",
        "mean_overlap": "Mean % Overlap Invalid Trials",
        "std_overlap": "Std % Overlap Invalid Trials",
        "mean_baseline": "Mean % Baseline Invalid Trials",
        "std_baseline": "Std % Baseline Invalid Trials",
        "mean_gap": "Mean % Gap Invalid Trials",
        "std_gap": "Std % Gap Invalid Trials"
    },
    inplace=True,
)

# Add the proportion of invalid trials to the document
doc.add_heading('Descrtion of invalid trial data', level=3)
doc = df_to_table(doc, proportion_invalid)


In [24]:
proportion_invalid

,NDD Group,Mean % Overlap Invalid Trials,Mean % Baseline Invalid Trials,Mean % Gap Invalid Trials,summ_overlap,summ_baseline,summ_gap
0,ndd,62.02,57.12,67.99,107.0,111.0,119.0
1,no_ndd,69.05,57.56,52.95,34.0,60.0,38.0


In [25]:
# Save the document as a copy with dat to archive 
doc.save(f"C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/outputs/word_document/archive/gap_et_preprocessing_results_{date}.docx")
# Print the path to the saved document
doc.save(f"C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/outputs/word_document/gap_et_preprocessing_results.docx")